<a href="https://colab.research.google.com/github/karthiksagarN/Mistral-7B-Instruct-v0.1-FineTuned/blob/main/Mistral_finetuned_chtabot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qsd9yzy6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qsd9yzy6
  Resolved https://github.com/huggingface/transformers to commit d9850abd407821fb9f4ad5aabb206adad3610f75
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.7/417

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os

data = load_dataset("tatsu-lab/alpaca", split="train")
data_df = data.to_pandas()
data_df = data_df[:10000]
data_df["text"] = data_df.apply(lambda row: "###Human: " + row["instruction"] + " " + row["input"] + "###Assistant: " + row["output"], axis=1)

# data_df["text"] = data_df[["instruction", "output"]].apply(lambda x: "###Human: " + x["instruction"] + "###Assistant: " + x["output"])


data = Dataset.from_pandas(data_df)

tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token

quantization_config_loading = GPTQConfig(bits=4, tokenizer=tokenizer, disable_exllama=True)

model = AutoModelForCausalLM.from_pretrained(
                                            "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                                            quantization_config=quantization_config_loading,
                                            device_map="auto"
                                            )

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = ["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir = "Mistral-Alpaca-Finetuned",
    hub_model_id="karthiksagarn/Mistral-Alpaca-Finetuned",
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 1,
    optim = "paged_adamw_32bit",
    learning_rate = 2e-4,
    lr_scheduler_type = "cosine",
    save_strategy = "epoch",
    logging_steps = 100,
    num_train_epochs = 1,
    # max_steps = 250,
    fp16 = True
    )

trainer = SFTTrainer(
    model = model,
    train_dataset = data,
    peft_config = peft_config,
    dataset_text_field = "text",
    max_seq_length = 512,
    tokenizer = tokenizer,
    args = training_args,
    packing=False
    )


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [2]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.448800
200,1.280600
300,1.291100
400,1.234700
500,1.240900
600,1.239900
700,1.241200
800,1.244400
900,1.225300
1000,1.231600


TrainOutput(global_step=1250, training_loss=1.2601404357910155, metrics={'train_runtime': 4003.3845, 'train_samples_per_second': 2.498, 'train_steps_per_second': 0.312, 'total_flos': 1597508779180032.0, 'train_loss': 1.2601404357910155, 'epoch': 1.0})

In [3]:
trainer.push_to_hub()

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1713496230.1178ee48fb6a.5279.0:   0%|          | 0.00/8.30k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/karthiksagarn/Mistral-Alpaca-Finetuned/commit/c033a1e3e79277f63f5e14a2db46a08dba411814', commit_message='End of training', commit_description='', oid='c033a1e3e79277f63f5e14a2db46a08dba411814', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
! cp -r /content/Mistral-Alpaca-Finetuned /content/drive/MyDrive/

### INFERENCE

In [7]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Mistral-Alpaca-Finetuned")

inputs = tokenizer("###Human: Hi what is your name? ###Assistant:", return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Mistral-Alpaca-Finetuned",
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="cuda"
    )
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id
)

import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time() - st_time)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4371: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


###Human: Hi what is your name? ###Assistant: My name is Mistral. How can I help you today? ��������������������������������������������������������������������������������������
5.8314549922943115


### Evaluation Metrics

In [11]:
# Evaluation
eval_data = load_dataset("tatsu-lab/alpaca", split="validation")
eval_data = data.to_pandas()
eval_data = data_df[:10000]

eval_results = trainer.evaluate(eval_dataset=eval_data)

# Print evaluation results
print(eval_results)

In [14]:
import pandas as pd

train_output = {
    "global_step": [1250],
    "training_loss": [1.2601404357910155],
    "train_runtime": [4003.3845],
    "train_samples_per_second": [2.498],
    "train_steps_per_second": [0.312],
    "total_flos": [1597508779180032.0],
    "train_loss": [1.2601404357910155],
    "epoch": [1.0]
}

train_output_table = pd.DataFrame(train_output)

train_output_table


,global_step,training_loss,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss,epoch
0,1250,1.26014,4003.3845,2.498,0.312,1.597509e+15,1.26014,1.0
